In [1]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from gauge_utils import SimpleProblem

import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr, LinExpr

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

num_var = 100
num_ineq = 50
num_eq = 50
num_examples = 10
print(num_ineq, num_eq)
np.random.seed(17)
Q = np.diag(np.random.random(num_var))
p = np.random.random(num_var)
A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
X = np.random.uniform(-1, 1, size=(num_examples, num_eq))
G = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
h = np.sum(np.abs(G@np.linalg.pinv(A)), axis=1)
data = SimpleProblem(Q, p, A, G, h, X)
data.calc_Y()
print(len(data.Y))
data.remove_no_ip()
print(len(data.Y))


50 50
running osqp
10
4


In [47]:
def get_interior_point(data, x): 
    tmp_h = data.h_np - data.G_np[:, data.other_vars] @ data._A_other_inv.detach().cpu().numpy() @ x
    tmp_G = data.G_np[:, data.partial_vars] - data.G_np[:, data.other_vars] @ data._A_other_inv.detach().cpu().numpy() @ data.A_np[:, data.partial_vars]
    M = 1e4
    
    model = gp.Model("qp")
    model.setParam('OutputFlag', 1)

    n_vars = len(data.partial_vars)
    vars = []
    for i in range(n_vars):
        vars.append(model.addVar(lb=-np.infty, ub=np.infty, vtype=GRB.CONTINUOUS, name=f"yp_{i}"))
    
    ya = model.addVar(lb=-np.infty, ub=np.infty, vtype=GRB.CONTINUOUS, name="ya")

    obj = LinExpr()

    obj.add(M * ya)
    model.setObjective(obj, GRB.MINIMIZE)

    for i in range(tmp_G.shape[0]):
        for j in range(n_vars):
            if tmp_G[i, j] != 0:
                expr = LinExpr()
                expr.add(vars[j] * tmp_G[i, j])
                model.addConstr(expr <= tmp_h[i] + ya)
    
    return model